In [26]:
from utils.args import models_list, graph_lambda_methods
import numpy as np
def adjust_lambda(epoch, num_epochs, lambda_0, lambda_n, lambda_epoch_max, method='cos'):
    assert method in graph_lambda_methods
    assert epoch < num_epochs

    index = graph_lambda_methods.index(method)
    if index == 0:
        k = np.log(10 * lambda_0 / num_epochs)
        return lambda_0 * np.exp(-k * epoch / num_epochs)
    elif index == 1:
        return lambda_0 * np.cos(np.pi / 2 / lambda_epoch_max * epoch)
    else:
        raise IndexError("请选择正确的 adjency matrix lambda 策略")


In [6]:
adjust_lambda()